In [1]:
import cv2
from cv2 import aruco
import numpy as np
import msgpack as mp
import msgpack_numpy as mpn
import os
import matplotlib.pyplot as plt

In [2]:
_pth = os.getcwd()
_pth = os.path.dirname(_pth)
_parent_folder = "validation"
_calib_folder_name = "calibration_charuco"


_base_pth = os.path.join(_pth,"recorded_data",_parent_folder)

_webcam_calib_folder = os.path.join(_pth,"recorded_data",_parent_folder,_calib_folder_name)
_webcam_calib_video = os.path.join(_webcam_calib_folder,"webcam_color.msgpack")
_webcam_calib_folder = os.path.join(_webcam_calib_folder)
_webcam_calib_pth = os.path.join( _webcam_calib_folder, "webcam_calibration.msgpack")

In [3]:
ARUCO_PARAMETERS = aruco.DetectorParameters()
ARUCO_DICT = aruco.getPredefinedDictionary(aruco.DICT_ARUCO_ORIGINAL)
squareLength = 0.05
markerLength = 0.03

board = aruco.CharucoBoard(
        size= [4,5],
        squareLength = squareLength,
        markerLength = markerLength,
        dictionary=ARUCO_DICT)

detector = aruco.CharucoDetector(board)


In [4]:
_video_pth = _webcam_calib_video
_video_file = open(_video_pth, "rb")
_video_data = mp.Unpacker(_video_file, object_hook=mpn.decode)
_video_length = 0

for _frame in _video_data:
    _video_length += 1

_video_file.close()

In [5]:
_video_pth = _webcam_calib_video
_video_file = open(_video_pth, "rb")
_video_data = mp.Unpacker(_video_file, object_hook=mpn.decode)

marker_corners, detected_ids, image_points, object_points = [], [], [], []
num_of_corners = []

first = True
counter = []
rnd = np.random.choice(_video_length, 200, replace=False)
for idx, _frame in enumerate(_video_data):
    
    if idx in rnd:
        _frame = cv2.cvtColor(_frame, cv2.COLOR_BGR2GRAY)

        current_corners, current_ids, current_imgpoints, current_objpoints = detector.detectBoard(_frame)
        
        if current_corners is None:
            continue
                    
        if len(current_corners) != 12:
            continue
        
        current_imgpoints = np.array(current_imgpoints)
        current_objpoints, current_imgpoints = board.matchImagePoints(current_corners, current_ids,current_objpoints,current_imgpoints)
        if first == True:
            corners_list = current_corners
            id_list = current_ids
            imagepoints_list = current_imgpoints
            objectpoints_list = current_objpoints
            first = False
        else:
            corners_list = np.vstack((corners_list, current_corners))
            id_list = np.vstack((id_list,current_ids))
            imagepoints_list = np.vstack((imagepoints_list, current_imgpoints))
            objectpoints_list = np.vstack((objectpoints_list, current_objpoints))
        counter.append(len(current_ids))
    
        marker_corners.append(current_corners)
        detected_ids.append(current_ids)
        image_points.append(current_imgpoints)
        object_points.append(current_objpoints)

_video_file.close()

counter = np.array(counter)

In [6]:
print(len(image_points), len(object_points), len(marker_corners), len(detected_ids))

175 175 175 175


In [7]:
marker_corners = np.array(marker_corners)
detected_ids = np.array(detected_ids)
image_points = np.array(image_points)
object_points = np.array(object_points)

In [8]:
mtx2 = np.zeros((3, 3))
dist2 = np.zeros((1, 5))

In [9]:
result = cv2.calibrateCamera(object_points, image_points, _frame.shape[:2], mtx2, dist2)

In [10]:
mtx2 = result[1]
dist2 = result[2]

In [11]:
with open(os.path.join(os.path.dirname(_webcam_calib_video), "webcam_calibration_charuco.msgpack"), "wb") as p:
    pckd = mp.packb((mtx2, dist2), default=mpn.encode)
    p.write(pckd)
    p.close()